## Analisis para la prediccion de Tiempo de Vuelta usando Regresion Lineal Multiple


`Melbourne` (Gran Premio de Australia) es un circuito urbano ubicado en Albert Park, `Melbourne`. Con una longitud de 5303 metros, se caracteriza por ser una pista mixta que combina secciones de alta velocidad con curvas técnicas. Aunque es un circuito que se corre en sentido horario, las zonas más difíciles son aquellas con múltiples cambios de dirección, lo que exige un alto nivel de control y precisión.

El clima en `Melbourne` es impredecible, con cambios repentinos de temperatura y posibles lluvias que complican las estrategias de los equipos. Las curvas de alta velocidad y las rectas relativamente cortas hacen que las paradas en boxes sean cruciales para los pilotos. Por lo tanto, es esencial tener un buen manejo de los neumáticos, especialmente en las zonas donde el asfalto es más abrasivo. El desempeño en la frenada y las aceleraciones de las curvas 1 y 3 son clave para conseguir tiempos rápidos.

![Melbourne](../img/melbourne.jpg)

Hagamos entonces un estudio de un modelo de regresión lineal múltiple con variable dependiente `FinalRaceTime` en el circuito de Melbourne


In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector

def load_data(filepath, circuit_name):
    data = pd.read_csv(filepath)
    circuit_data = data[data['Circuit'] == circuit_name].copy()
    if circuit_data.empty:
        raise ValueError(f"No data found for circuit {circuit_name}")
    return circuit_data

def select_features(data, target, initial_features):
    features = [f for f in initial_features if f in data.columns]
    if target not in data.columns:
        raise ValueError(f"Target column '{target}' not found in the dataset")
    return features

def preprocess_data(data, features, target):
    for feature in features:
        data[feature] = pd.to_numeric(data[feature], errors='coerce')
    data = data.dropna(subset=features + [target])
    return data

def train_model(X, y):
    
    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)



    # Create and train the model
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)

    # Print the equation of the hyperplane
    coefficients = model.coef_
    intercept = model.intercept_

    #print("\nEquation of the hyperplane:")
    equation = f"FinalRiceTime = {intercept:.2f}"
    for feature, coef in zip(features, coefficients):
        equation += f" + ({coef:.2f} * {feature})"
    #print(equation)

    # Calculate R-squared
    r_squared = model.score(X_test_scaled, y_test)
    
   
# Agregar constante para la intersección
    X_train_const = sm.add_constant(X_train_scaled)


# Ajustar el modelo de regresión con statsmodels
    model_sm = sm.OLS(y_train, X_train_const).fit()

# Mostrar los p-values de cada coeficiente
    #print(model_sm.summary())

# Create a DataFrame for coefficients and p-values
    summary_df = pd.DataFrame({
    'Feature': model_sm.params.index,
    'Coefficient': model_sm.params.values,
    'P-value': model_sm.pvalues.values
    })

# Filter to keep only p-values greater than 0.05
    summary_df = summary_df[summary_df['P-value'] > 0.05]

# Sort the DataFrame by p-value in descending order
    summary_df = summary_df.sort_values(by='P-value', ascending=False)


# Print the sorted DataFrame
    # print("\nSorted Coefficients and P-values (from highest to lowest p-value):")
    # print(summary_df)
    return summary_df



# Ejecutar el pipeline
data = load_data("formula1_interlagos_df_final.csv", "Melbourne")
features = ['MaxSpeed', 'DriverSkill', 'Age', 'PitStopTime', 'ReactionTime',
                    'FinalPosition', 'Experience', 'DNF', 'Points', 'Overtakes', 'TyreWear',
                    'CarPerformance', 'TrackFamiliarity', 'FuelConsumption', 'DownforceLevel',
                    'TrackTemperature', 'WeatherCondition_Mixed', 'WeatherCondition_Wet',
                    'TyreCompound_Medium', 'TyreCompound_Soft', 'TrackGrip_Low', 'TrackGrip_Medium']

target = 'FinalRaceTime'
features = select_features(data, target, features)
data = preprocess_data(data, features, target)
X, y = data[features], data[target]

summary_df = train_model(X, y)

while len(summary_df)>0:
    top_feature = str(summary_df.iloc[0]['Feature']) # Primer variable (con mayor p-valor)
    
    #print(top_feature[1:])
    
    del features[int(top_feature[1:])-1]
    #print(f"Variable {top_feature} eliminada de 'features'.")
    #print(f"Lista de variables restantes: {features}")
    
    features = select_features(data, target, features)
    data = preprocess_data(data, features, target)
    X, y = data[features], data[target]
    summary_df = train_model(X, y)
    
print(features)

['MaxSpeed', 'Age', 'FinalPosition', 'Points', 'Overtakes', 'TyreWear', 'WeatherCondition_Mixed', 'TrackGrip_Low']


Aquí hemos podido ver que aplicamos un método de selección de características basado en `backward elimination`. En cada iteración, eliminamos la variable con el mayor p-valor (es decir, la menos significativa), y repetimos el proceso hasta que todas las variables restantes tienen un p-valor menor a 0.05, lo que sugiere que son estadísticamente significativas para predecir el tiempo final de la carrera. Finalmente, nos hemos quedado con un conjunto reducido de variables independientes que tienen una mayor relación con el objetivo de la predicción, FinalRaceTime, lo que mejora la precisión del modelo.

In [7]:

def train_model2(X, y):
    
    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)



    # Create and train the model
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)

    # Print the equation of the hyperplane
    coefficients = model.coef_
    intercept = model.intercept_

    #print("\nEquation of the hyperplane:")
    equation = f"FinalRiceTime = {intercept:.2f}"
    for feature, coef in zip(features, coefficients):
        equation += f" + ({coef:.2f} * {feature})"
    #print(equation)

    # Calculate R-squared
    r_squared = model.score(X_test_scaled, y_test)
    
   
# Agregar constante para la intersección
    X_train_const = sm.add_constant(X_train_scaled)


# Ajustar el modelo de regresión con statsmodels
    model_sm = sm.OLS(y_train, X_train_const).fit()

# Mostrar los p-values de cada coeficiente
    print(model_sm.summary())


X, y = data[features], data[target]
summary_df = train_model2(X, y)


                            OLS Regression Results                            
Dep. Variable:          FinalRaceTime   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.880
Method:                 Least Squares   F-statistic:                     29.53
Date:                Tue, 04 Mar 2025   Prob (F-statistic):           2.52e-10
Time:                        16:20:10   Log-Likelihood:                -80.985
No. Observations:                  32   AIC:                             180.0
Df Residuals:                      23   BIC:                             193.2
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        267.6977      0.634    422.321      0.0

### Análisis del Modelo de Regresión OLS

El modelo de regresión lineal ajustado tiene como variable dependiente FinalRaceTime y utiliza las siguientes variables predictoras:

- MaxSpeed
- Age
- FinalPosition
- Points
- Overtakes
- TyreWear
- WeatherCondition_Mixed
- TrackGrip_Low

#### Evaluación General del Modelo

- $R^2$ = 0.911 -> Indica que el 91.1% de la variabilidad en el `FinalRaceTime` puede ser explicada por las variables predictoras del modelo.
- $R^2$ ajustado = 0.880 -> Penaliza el $R^2$  por el número de predictores. Aunque es menor, sigue siendo alto, lo que sugiere un buen ajuste del modelo.

El modelo explica una gran parte de la variabilidad en los tiempos de carrera. Un

$R^2$ mayor a 0.8 sugiere un buen modelo en términos de ajuste.

#### Significancia Global del Modelo

``Prueba F-Statistic`` : prueba estadística utilizada para evaluar la **significancia global** del modelo de regresión. Responde a las pregunta:

> ¿Al menos una de las variables predictoras tiene un efecto significativo sobre la variable dependiente?

- F = 29.53 con p-value = 2.52e-10
- Esto indica que al menos una de las variables predictoras tiene un impacto significativo en FinalRaceTime.

#### Análisis de los Coeficientes y Significancia Individual

Cada coeficiente indica cuánto cambia FinalRaceTime cuando la variable predictora aumenta en una unidad, manteniendo las demás constantes.

| Variable                    | Coeficiente | p-valor | Interpretacion                                                                                                                 |
| --------------------------- | ----------- | ------- | ------------------------------------------------------------------------------------------------------------------------------ |
| Intercept (Constante)       | 267.70      | 0.000   | Tiempo base sin efectos de las variables predictoras                                                                           |
| MaxSpeed (x1)               | -3.85       | 0.000   | A mayor velocidad máxima, menor tiempo de carrera                                                                             |
| Age (x2)                    | -2.59       | 0.001   | A mayor edad, menor tiempo de carrera (pilotos más experimentados pueden ser más rápidos)                                   |
| FinalPosition (x3)          | 13.95       | 0.000   | A medida que la posición final aumenta, el tiempo de carrera es mayor (pilotos más lentos terminan en posiciones más bajas) |
| Points (x4)                 | 8.88        | 0.000   |                                                                                                                                |
| Overtakes (x5)              | -2.77       | 0.008   | Más adelantamientos reducen el tiempo de carrera (pilotos agresivos ganan tiempo)                                             |
| TyreWear (x6)               | -5.28       | 0.000   | Más desgaste de neumáticos, menor tiempo (posiblemente estrategia agresiva con neumáticos blandos)                          |
| WeatherCondition_Mixed (x7) | 4.24        | 0.000   | Condiciones climáticas mixtas aumentan el tiempo de carrera                                                                   |
| TrackGrip_Low (x8)          | 4.50        | 0.000   | Bajo agarre de pista aumenta el tiempo de carrera                                                                              |


**Conclusión:**

- Todos los **p-valores** son  **< 0.05** , lo que significa que  **todas las variables son significativas** .
- Los signos de los coeficientes tienen  **interpretaciones lógicas** , lo que refuerza la validez del modelo.
- El modelo parece ser  **robusto y bien especificado** .


## **Conclusión Final**

✅  **El modelo es sólido** , con un  **buen ajuste (R2=91.1R^2 = 91.1%**R**2**=**91.1)** , **predictores significativos** y  **una interpretación lógica de los coeficientes** .

📌  **Próximos Pasos** :

Antes de concluir la investigación, es importante validar los **supuestos del modelo** (normalidad, homoscedasticidad, independencia y multicolinealidad). Esto asegurará que los resultados sean confiables. 🚀

